## Model: Responsible Business Simulator (RBS)
In this notebook the core (calculation) steps of the RBS model are illustrated.

### (I): the RBS class
In order to work with the Responsible Business Simulator model, we will work with the ```ResponsibleSimulatorClass```. This class can be found in **core/tRBS.py**. This class takes three input values: (i) the name of the case, (ii) the format of the data (xlsx, json, csv) and (iii) the folder where data of the case can be found.

In [1]:
from pathlib import Path
from core.trbs import ResponsibleBusinessSimulator

name = 'Beerwiser'
file_format = 'xlsx'
path = Path.cwd() / 'data' / 'beerwiser'

case = ResponsibleBusinessSimulator(name, file_format, path)

Let's see how ```case``` looks currently:

In [2]:
print(case)

Case: Beerwiser (xlsx) 
Data location: /Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser 
Input data: 
 


Currently, it is rather empty. To get the input data, we will first need to ```build``` our case.

In [3]:
case.build()
print(case)

Creating 'Beerwiser'
Case: Beerwiser (xlsx) 
Data location: /Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser 
Input data: 
 configurations
	['use_theme_weights']

configuration_value
	[0]

key_outputs
	['Accidents reduction' 'Water use reduction' 'Production cost reduction']

key_output_unit
	['#/year' 'hl/year' '%']

key_output_theme
	['People' 'Planet' 'Profit']

themes
	['People' 'Planet' 'Profit']

key_output_minimum
	[nan nan nan]

key_output_maximum
	[nan nan nan]

key_output_monetary
	[0 0 0]

key_output_smaller_the_better
	[0 0 0]

key_output_linear
	[1 1 0]

key_output_automatic
	[1 1 1]

key_output_start
	[nan nan nan]

key_output_end
	[nan nan nan]

key_output_threshold
	[nan nan nan]

decision_makers_options
	['Focus on water recycling' 'Focus on training' 'Equal spread']

internal_variable_inputs
	['Invest in training of employees' 'Invest in water recycling']

decision_makers_option_value
	[[5.0e+004 2.5e+005]
 [2.0e-323 5.0e+004]
 [9.9e-324 1.5e

In [4]:
case.evaluate()


0. 'Accidents reduction %' = 'Invest in training of employees' Squeezed * ''


EvaluationError: Evaluation error: Squeezed * not available